In [1]:
import sys
print(sys.executable)

D:\Programme\Python3_8_MiniConda\envs\TF2_5\python.exe


In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("train_housePrice.csv")

def preprocessDataFrame(df):
    df['garage'] = df['garage'].fillna(0)
    df['yrremodeled'] = df['yrremodeled'].fillna(0)

    # replace string label by numberical label
    dic = {'Farm': 1, 'Ranch':2, 'Cottage': 3, 'Town':4, 'Detached':5, 'Bungalow':6}

    df['housetype'] = df['housetype'].map(dic)

preprocessDataFrame(df)


In [4]:
from sklearn.preprocessing import QuantileTransformer

trafo = QuantileTransformer()
target_trafo = QuantileTransformer()

cols = df.drop("price", axis=1).columns

X = trafo.fit_transform(df.drop("price", axis=1))
Y = target_trafo.fit_transform(df["price"].to_numpy().reshape((len(df), 1)))

In [5]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, random_state=42, test_size=0.2)

In [35]:
def Score(yPred, yTest, doReTransform=True):
    if doReTransform:
        yPred2 = target_trafo.inverse_transform(yPred)
        yTest2 = target_trafo.inverse_transform(yTest)
        return np.mean(np.abs((yPred2-yTest2)/yPred2))
        
    return np.mean(np.abs((yPred-yTest)/yPred))

In [25]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor



def buildCustomModel(input_shape, layer_number=[100, 100, 100], act_method=['relu', 'relu', 'relu', 'sigmoid'], batchnormalize= True, optimizer="adam", loss='mean_squared_error'):
    m_list = []
    m_list.append(layers.InputLayer(input_shape=input_shape))
    
    for i in range(len(layer_number)):
        m_list.append(layers.Dense(layer_number[i], activation=act_method[i]))
        if batchnormalize:
            m_list.append(layers.BatchNormalization(axis=1))
    m_list.append(layers.Dense(1, activation= act_method[-1]))

    model = keras.Sequential(m_list)
    model.compile(optimizer=optimizer, loss=loss)
    return model

param_grid = [
    {
    'input_shape': [len(cols)], 
    'layer_number': [[500, 200], [100, 100]],
    'act_method': [['relu', 'relu', 'sigmoid']], 
    'batchnormalize':[True, False],
    'optimizer': ['adam'], 
    'loss': ['mean_squared_error']
    }, 
    {
    'input_shape': [len(cols)], 
    'layer_number': [[500, 200, 100], [100, 100, 100]],
    'act_method': [['relu', 'relu', 'relu', 'sigmoid']], 
    'batchnormalize':[True, False],
    'optimizer': ['adam'], 
    'loss': ['mean_squared_error']
    }, 
    {
    'input_shape': [len(cols)], 
    'layer_number': [[500, 200, 100, 50], [100, 100, 100, 100]],
    'act_method': [['relu', 'relu', 'relu', 'relu', 'sigmoid']], 
    'batchnormalize':[True, False],
    'optimizer': ['adam'], 
    'loss': ['mean_squared_error']
    }, 
    {
    'input_shape': [len(cols)], 
    'layer_number': [[500, 200, 100, 50, 25], [100, 100, 100, 100, 100]],
    'act_method': [['relu', 'relu', 'relu', 'relu', 'relu', 'sigmoid']], 
    'batchnormalize':[True, False],
    'optimizer': ['adam'], 
    'loss': ['mean_squared_error']
    }
    ]


model = KerasRegressor(build_fn=buildCustomModel, verbose=0)
gs = GridSearchCV(estimator=model,param_grid=param_grid, cv=2, n_jobs=None, scoring='neg_root_mean_squared_error', verbose=2)
fitted_model = gs.fit(Xtrain, Ytrain, epochs= 100)


Fitting 2 folds for each of 16 candidates, totalling 32 fits
[CV] END act_method=['relu', 'relu', 'sigmoid'], batchnormalize=True, input_shape=12, layer_number=[500, 200], loss=mean_squared_error, optimizer=adam; total time=  27.3s
[CV] END act_method=['relu', 'relu', 'sigmoid'], batchnormalize=True, input_shape=12, layer_number=[500, 200], loss=mean_squared_error, optimizer=adam; total time=  27.2s
[CV] END act_method=['relu', 'relu', 'sigmoid'], batchnormalize=True, input_shape=12, layer_number=[100, 100], loss=mean_squared_error, optimizer=adam; total time=  27.3s
[CV] END act_method=['relu', 'relu', 'sigmoid'], batchnormalize=True, input_shape=12, layer_number=[100, 100], loss=mean_squared_error, optimizer=adam; total time=  27.7s
[CV] END act_method=['relu', 'relu', 'sigmoid'], batchnormalize=False, input_shape=12, layer_number=[500, 200], loss=mean_squared_error, optimizer=adam; total time=  14.4s
[CV] END act_method=['relu', 'relu', 'sigmoid'], batchnormalize=False, input_shape=

In [26]:
fitted_model.estimator.get_params()

{'verbose': 0,
 'build_fn': <function __main__.buildCustomModel(input_shape, layer_number=[100, 100, 100], act_method=['relu', 'relu', 'relu', 'sigmoid'], batchnormalize=True, optimizer='adam', loss='mean_squared_error')>}

In [34]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

model = keras.Sequential([
    layers.InputLayer(input_shape=len(cols)),
    layers.Dense(100, activation='relu'),
    layers.BatchNormalization(axis=1),
    layers.Dense(100, activation='relu'),
    layers.BatchNormalization(axis=1),
    layers.Dense(100, activation='relu'),
    layers.BatchNormalization(axis=1),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(Xtrain, Ytrain, batch_size=1000, epochs=500, verbose=1)

Epoch 1/500
4/4 [==============================] - 1s 6ms/step - loss: 0.1266
Epoch 2/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0404
Epoch 3/500
4/4 [==============================] - 0s 6ms/step - loss: 0.0311
Epoch 4/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0253
Epoch 5/500
4/4 [==============================] - 0s 6ms/step - loss: 0.0206
Epoch 6/500
4/4 [==============================] - 0s 6ms/step - loss: 0.0165
Epoch 7/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0139
Epoch 8/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0114
Epoch 9/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0092
Epoch 10/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0082
Epoch 11/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0074
Epoch 12/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0068
Epoch 13/500
4/4 [==============================] - 0s 6ms/st

In [36]:
pred = model.predict(Xtest)
print(Score(pred, Ytest))

0.12374066105101096
